#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [19]:
import pandas as pd
import pymongo

In [20]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [21]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [22]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [94]:
df_ks['launched'] = pd.to_datetime(df_ks['launched'],errors='coerce')
df_ks['pledged'] = pd.to_numeric(df_ks['pledged'])
df_ks['goal'] = pd.to_numeric(df_ks['goal'])
df_ks = df_ks.dropna(subset=['launched'])

In [95]:
df_ks = df_ks.rename(columns={"ID": "_id"})

In [96]:
df_ks.dtypes


_id                          int64
name                        object
category                    object
main_category               object
currency                    object
deadline                    object
goal                       float64
launched            datetime64[ns]
pledged                    float64
state                       object
backers                     object
country                     object
usd pledged                 object
usd_pledged_real           float64
dtype: object

In [121]:
df_ks.country.unique()

array(['US', 'CA', 'HK', 'BE', 'GB', 'NO', 'NL', 'N,0"', 'AU', 'CH', 'SE',
       'AT', 'NZ', 'MX', 'FR', 'DK', 'IT', 'ES', 'DE', 'IE', 'SG', 'LU',
       'JP'], dtype=object)

### Importer les données

In [46]:
documents = df_ks.to_dict(orient = 'records')

In [47]:
collection.insert_many(documents)

In [28]:
collection.find_one()

{'_id': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': '2011-09-16',
 'goal': 2000.0,
 'launched': datetime.datetime(2011, 8, 17, 6, 31, 31),
 'pledged': 1145.0,
 'state': 'canceled',
 'backers': 24,
 'country': 'US',
 'usd pledged': 1145.0,
 'usd_pledged_real': 1145.0}

## Question 1  

In [49]:
max_pledged = collection.aggregate([{"$group" : {"_id" : "$name", "pledged" : {"$max" : "$pledged"}}},{
                                            "$sort": {"pledged" : -1}}])
list(max_pledged)[0:5]

[{'_id': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'pledged': 13285226.36},
 {'_id': 'Pebble 2, Time 2 + All-New Pebble Core', 'pledged': 12779843.49},
 {'_id': 'Expect the Unexpected. digiFilmï¿½ Camera by YASHICA',
  'pledged': 10035296.0},
 {'_id': 'OUYA: A New Kind of Video Game Console', 'pledged': 8596474.58},
 {'_id': 'The Everyday Backpack, Tote, and Sling', 'pledged': 6565782.5}]

## Question 2

In [117]:
successful_cursor = collection.find({"state": "successful"})

In [118]:
compteur=0
for document in successful_cursor :
    compteur+=1
print(compteur)

53040


## Question 3

In [119]:
number_project_category = collection.aggregate([{"$group" : {"_id" : "$category", "count" : {"$sum" : 1}}},{
                                            "$sort": {"count" : -1}}])
list(number_project_category)

[{'_id': 'Product Design', 'count': 8886},
 {'_id': 'Documentary', 'count': 6498},
 {'_id': 'Music', 'count': 6229},
 {'_id': 'Tabletop Games', 'count': 5581},
 {'_id': 'Shorts', 'count': 4857},
 {'_id': 'Video Games', 'count': 4799},
 {'_id': 'Food', 'count': 4612},
 {'_id': 'Film & Video', 'count': 4003},
 {'_id': 'Fiction', 'count': 3703},
 {'_id': 'Nonfiction', 'count': 3390},
 {'_id': 'Fashion', 'count': 3379},
 {'_id': 'Art', 'count': 3358},
 {'_id': 'Apparel', 'count': 2827},
 {'_id': 'Theater', 'count': 2786},
 {'_id': 'Rock', 'count': 2707},
 {'_id': 'Technology', 'count': 2690},
 {'_id': "Children's Books", 'count': 2686},
 {'_id': 'Apps', 'count': 2535},
 {'_id': 'Publishing', 'count': 2332},
 {'_id': 'Webseries', 'count': 2316},
 {'_id': 'Photography', 'count': 2239},
 {'_id': 'Indie Rock', 'count': 2192},
 {'_id': 'Narrative Film', 'count': 2099},
 {'_id': 'Web', 'count': 2017},
 {'_id': 'Comics', 'count': 1931},
 {'_id': 'Crafts', 'count': 1834},
 {'_id': 'Country & Folk'

## Question 4

In [126]:
import datetime
d = datetime.datetime(2016, 0o1, 0o1, 0o1)
nb_projetfr = collection.aggregate([{"$match":{"country":"FR","launched" : {'$lt': d}}},{"$group" : {"_id" : "$name", "date" : {"$max" : "$launched"}}}])
len(list(nb_projetfr))

330

#### Question 5

In [107]:
american_project = collection.aggregate([{"$match":{"country":"US","goal" : {'$gt': 200000}}},{"$group" : {"_id" : "$name", "demande" : {"$max" : "$goal"}}},
                                        {
                                            "$sort": {"demande" : -1}}])
list(american_project)

[{'_id': 'Kybernan Holographic Gaming Network', 'demande': 100000000.0},
 {'_id': 'The Time Jumper', 'demande': 100000000.0},
 {'_id': 'The Scariest Movie Ever Created', 'demande': 100000000.0},
 {'_id': 'The Throne : Fall of Lucifer', 'demande': 100000000.0},
 {'_id': 'The Return Of The Bell Witch Movie', 'demande': 100000000.0},
 {'_id': 'The Best Kickstarter Video Ever', 'demande': 73000000.0},
 {'_id': 'The Cultivator Episode 1 Lord Of The Gorgs', 'demande': 50000000.0},
 {'_id': 'Save the Williamsburg', 'demande': 40000000.0},
 {'_id': 'Gouddaa series (Canceled)', 'demande': 40000000.0},
 {'_id': 'equal rights for all campaign', 'demande': 35000000.0},
 {'_id': 'Intelligent Design: The Series', 'demande': 33000000.0},
 {'_id': 'Help erase the national debt of the USA', 'demande': 21474836.47},
 {'_id': 'Flat Creek Agri-Tourism and Outdoor Family Gaming',
  'demande': 16610000.0},
 {'_id': 'Jesus Vs The Dragons', 'demande': 10000000.0},
 {'_id': 'The Confidant', 'demande': 10000000

## Question 6 

In [115]:
sport = collection.find({"name": {"$regex" : ".*Sport.*"}})


In [116]:
compteur2=0
for document in sport :
    compteur2+=1
print(compteur2)

323
